<a href="https://colab.research.google.com/github/harshadj1611/IMDB-Reviews---Sentimental-Analysis/blob/main/IMDB_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading the library wordninja

In [20]:
!pip install wordninja

     |████████████████████████████████| 541 kB 6.3 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541551 sha256=0b9e9ea1991fb3fccbaff691edf63bcf05e746627e2fefc921c426b6e75f034e
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja


## Importing required libraries

In [14]:

import pandas as pd 
import numpy as np
import re
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk.stem import WordNetLemmatizer

## The datasets we are going to use (Downloaded from Kaggle)

In [3]:
train = pd.read_csv("/content/Train.csv")
test = pd.read_csv("/content/Test.csv")

In [4]:
train.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [8]:
test.head()

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0


## Checking if there are null values

In [6]:
train.isnull().sum()

text     0
label    0
dtype: int64

In [7]:
test.isnull().sum()

text     0
label    0
dtype: int64

## Cleaning the data 

In [9]:
## Converting every word in lower text
train['text'] = train['text'].apply(lambda x : str.lower(x))
test['text'] = test['text'].apply(lambda x : str.lower(x))

In [10]:
## Cleaning if any url present in the dataset
def clean_url(text):
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    text = text.strip()
    return text

In [11]:
train['text'] = train['text'].apply(lambda x : clean_url(x))
test['text'] = test['text'].apply(lambda x : clean_url(x))

In [15]:
## Cleaning the dataset further
def clean(text):
    text = re.sub("(@[A-Za-z0-9_]+)","", text) #remove mentions or tags
    text = re.sub('<br\s?\/>|<br>', " ", text) # remove br tags
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    
    text = re.sub(" +", " ", text) #remove extra white spaces
    text = re.sub("[0-9]+", "", text) #remove in numeric data
    text = re.sub("[^A-Za-z0-9_. ]+","",text) #remove any other characters other than A-Z a-z 0-9. 
    return text    

In [16]:
train['text'] = train['text'].apply(lambda x : clean(x))
test['text'] = test['text'].apply(lambda x : clean(x))

In [17]:
## Removing blank spaces
def blank_space(text):
    text = ' '.join(text.split())
    
    
    return text

In [18]:
train['text'] = train['text'].apply(lambda x : blank_space(x))
test['text'] = test['text'].apply(lambda x : blank_space(x))

In [21]:
## If two English words get joined from the above step, we will seperate them in this step by using wordninja package
import wordninja 
def ninja(text):
    text = wordninja.split(text)
    text = " ".join(text)
    return text


In [22]:
train['text'] = train['text'].apply(lambda x : ninja(x))
test['text'] = test['text'].apply(lambda x : ninja(x))

## Removing stop words

In [23]:
nltk.download('stopwords')
def remove_stopwords(text):
    stopword = stopwords.words('english')
    
    text = [word for word in text.split() if not word in stopword]
    text = " ".join(text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
train['text'] = train['text'].apply(lambda x : remove_stopwords(x))
test['text'] = test['text'].apply(lambda x : remove_stopwords(x))

In [29]:
## tokenizer 
nltk.download('punkt')
def tokens(text):
    text = word_tokenize(text)
    text = " ".join(text)
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [30]:
train['text'] = train['text'].apply(lambda x : tokens(x))
test['text'] = test['text'].apply(lambda x : tokens(x))

In [31]:
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download('state_union')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Unzipping corpora/state_union.zip.


True

## Function to get Parts Of Speech of the words

In [32]:
## Creating a function to get the parts of speech of the words.
## It gives us the context in which the word has been used.
## For example: 'watch' can be used as a verb but 'watch' as in wrist watch is a noun

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import state_union

training = state_union.raw("2005-GWBush.txt")
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer(training)

def get_word_pos(word):
    
    sample = word
    
    tokenized = tokenizer.tokenize(sample)
#     print(tokenized)
    
    
    for i in tokenized:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
#         print(tagged)
        
    for letter in tagged:
    #print(letter[1])
        if letter[1].startswith('J'):
            return wordnet.ADJ
        elif letter[1].startswith('V'):
            return wordnet.VERB
        elif letter[1].startswith('N'):
            return wordnet.NOUN
        elif letter[1].startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

## Performing process of lemmatization

In [33]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    
    lemmatized_text = []
    
    tokens = word_tokenize(text)
#     print(tokens)
    
    for word in tokens:
        
        lemmatized_text.append(lemmatizer.lemmatize(word, get_word_pos(word)))

    return " ".join(lemmatized_text)

In [34]:
train['text'] = train['text'].apply(lambda x : lemmatize(x))
test['text'] = test['text'].apply(lambda x : lemmatize(x))

In [35]:
x_train = train['text'].copy()
x_test = test['text'].copy()

y_train = train['label'].copy()
y_test = test['label'].copy()

In [36]:
x_train.head()

0    grow b watch love thunderbird mate school watc...
1    put movie dvd player coke chip expectation hop...
2    people know particular time past like feel nee...
3    even though great interest biblical movie bore...
4    im die hard dad army fan nothing ever change g...
Name: text, dtype: object

In [37]:
print("Train: ",x_train.shape,y_train.shape,"Test: ",(x_test.shape,y_test.shape))

Train:  (40000,) (40000,) Test:  ((5000,), (5000,))


## Implementing TfidVectorizer

In [38]:
## Term frequency-inverse document frequency is a text vectorizer that transforms the text into a usable vector
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=4000, min_df=1, max_df=0.9, ngram_range=(1,1))  
X_train = tfidfconverter.fit_transform(x_train)
X_test = tfidfconverter.transform(x_test)

## Implementing classification models

In [39]:
## The objective of a Linear SVC (Support Vector Classifier) is to fit to the data you provide, returning a "best fit" hyperplane that divides, or categorizes, your data
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

In [40]:
clf.fit(X_train,y_train)

LinearSVC(random_state=0)

In [41]:
y_test_pred=clf.predict(X_test)

In [42]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
 
print(confusion_matrix(y_test,y_test_pred))  
print(classification_report(y_test,y_test_pred))  
print(accuracy_score(y_test, y_test_pred))

[[2172  323]
 [ 269 2236]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2495
           1       0.87      0.89      0.88      2505

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000

0.8816


In [43]:
## A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting

from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)  
text_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

In [44]:
predictions = text_classifier.predict(X_test)
 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
 
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[2130  365]
 [ 364 2141]]
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      2495
           1       0.85      0.85      0.85      2505

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000

0.8542


In [45]:
## Logistic regression is a process of modeling the probability of a discrete outcome given an input variable

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=3500,solver='saga')

In [46]:
clf.fit(X_train,y_train)

LogisticRegression(max_iter=3500, solver='saga')

In [68]:
y_test_pred=clf.predict(X_test)

In [47]:
print(confusion_matrix(y_test,y_test_pred))  
print(classification_report(y_test,y_test_pred))  
print(accuracy_score(y_test, y_test_pred))

[[2172  323]
 [ 269 2236]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2495
           1       0.87      0.89      0.88      2505

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000

0.8816
